In [1]:
import numpy as np
import h5py
import sys


sys.path.append(".")
dirname = "./bgw_files/k-2x2x2/"

In [2]:
# epsinp and sigmain data.
from qtm.interfaces.bgw.epsinp import Epsinp
epsinp = Epsinp.from_epsilon_inp(filename=dirname+'epsilon.inp')

from qtm.interfaces.bgw.sigmainp import Sigmainp
sigmainp = Sigmainp.from_sigma_inp(filename=dirname+'sigma.inp')

In [3]:
from qtm.interfaces.bgw import inp
from qtm.interfaces.bgw.wfn2py import wfn2py

# Read wfndata
wfndata = wfn2py(dirname + "WFN.h5")
wfnqdata = wfn2py(dirname + "WFNq.h5")

# Read rho
rho = inp.read_rho(dirname + "RHO")

# Read vxc 
vxc = inp.read_vxc(dirname + "vxc.dat") 

In [4]:
from qtm.gw.core import QPoints
from qtm.gw.epsilon import Epsilon

epsilon = Epsilon.from_data(wfndata=wfndata, wfnqdata=wfnqdata, epsinp=epsinp)

Vcoul calculation for qpts: 100%|██████████| 8/8 [00:00<00:00, 13876.94it/s]


In [5]:
from tqdm import trange
from qtm.gw.core import reorder_2d_matrix_sorted_gvecs, sort_cryst_like_BGW


def calculate_epsilon(numq=None, writing=False):
    epsmats = []
    if numq is None:
        numq = epsilon.qpts.numq

    for i_q in trange(0, numq, desc="Epsilon> q-pt index"):
        # Create map between BGW's sorting order and QTm's sorting order
        gkspc = epsilon.l_gq[i_q]
        
        if i_q == epsilon.qpts.index_q0:
            key = gkspc.g_norm2
        else:
            key = gkspc.gk_norm2

        indices_gspace_sorted = sort_cryst_like_BGW(
            cryst=gkspc.g_cryst, key_array=key
        )
       

        # Calculate matrix elements
        M = next(epsilon.matrix_elements(i_q=i_q))

        # Calculate polarizability matrix (faster, but not memory-efficient)
        chimat = epsilon.polarizability(M)

        # Calculate polarizability matrix (memory-efficient)
        # chimat = epsilon.polarizability_active(i_q)

        # Calculate epsilon inverse matrix
        epsinv0 = epsilon.epsilon_inverse(i_q=i_q, polarizability_matrix=chimat, store=True)
        epsinv = reorder_2d_matrix_sorted_gvecs(epsinv0, indices_gspace_sorted)

        epsilon.l_epsinv[i_q] = epsinv

        
        # Compare the results with BGW's results
        if i_q == epsilon.qpts.index_q0:
            epsref = epsilon.read_epsmat(dirname + "eps0mat.h5")[0][0, 0]
            if writing:
                epsilon.write_epsmat(
                    filename="test/epsilon/eps0mat_qtm.h5", epsinvmats=[epsinv]
                )
        else:
            epsref = np.array(epsilon.read_epsmat(dirname + "epsmat.h5")[i_q - 1][0, 0])
            epsmats.append(epsinv)

        # Calculate stddev between reference and calculated epsinv matrices
        std_eps = np.std(epsref - epsinv) / np.sqrt(np.prod(list(epsinv.shape)))


        epstol = 1e-16
        if np.abs(std_eps) > epstol:
            print(f"Standard deviation exceeded {epstol} tolerance: {std_eps}, for i_q:{i_q}")


    if writing:
        epsilon.write_epsmat(filename="test/epsilon/epsmat_qtm.h5", epsinvmats=epsmats)


epsinp.no_min_fftgrid = True
epsilon = Epsilon.from_data(wfndata=wfndata, wfnqdata=wfnqdata, epsinp=epsinp)

calculate_epsilon()

Epsilon> q-pt index: 100%|██████████| 8/8 [00:00<00:00, 145.37it/s]

Standard deviation exceeded 1e-16 tolerance: 2.5494915419483335e-15, for i_q:0


In [6]:
from kernel import KernelMtxEl

q0val = [0.001, 0.001, 0.001]


In [7]:
kernelclass = KernelMtxEl.from_BGW(
    wfndata=wfndata,
    epsinp=epsinp,
    sigmainp=sigmainp,
    l_epsmats=epsilon.l_epsinv,
    q0val=q0val,
    parallel=False,
    num_bands_val=1,
    num_bands_con=1,
)

In [8]:
data = kernelclass.calc_kernel_mtxel()

In [9]:
exc = data["exc"]
print(exc.shape)
exc = exc[:, :, 0, 0, 0, 0]

for row in range(exc.shape[0]):
    row_data = []
    for col in range(exc.shape[1]):
        row_data.append(f"{exc[row, col]:.5f}")
    print(" | ".join(row_data))

(8, 8, 1, 1, 1, 1)
0.36344+0.00000j | -0.01658-0.06321j | -0.04075-0.01372j | -0.01385+0.02490j | -0.05664+0.08663j | 0.00407-0.01398j | 0.00559+0.02202j | 0.07646-0.00919j
-0.01658-0.06321j | 0.08180+0.00000j | 0.00674-0.01023j | 0.04141+0.03970j | 0.00900+0.00996j | 0.04827+0.02899j | -0.03789-0.00030j | -0.00662+0.04796j
-0.04075-0.01372j | 0.00674-0.01023j | 0.08179+0.00000j | -0.01028+0.05586j | -0.00305+0.01176j | 0.00157+0.03786j | -0.03091-0.04776j | 0.00138-0.00066j
-0.01385+0.02490j | 0.04141+0.03970j | -0.01028+0.05586j | 0.10747+0.00000j | 0.03780+0.00271j | 0.05244-0.01191j | -0.03904+0.03684j | 0.01896+0.02616j
-0.05664+0.08663j | 0.00900+0.00996j | -0.00305+0.01176j | 0.03780+0.00271j | 0.08180+0.00000j | 0.05384-0.01636j | -0.03883+0.04228j | -0.03016-0.03590j
0.00407-0.01398j | 0.04827+0.02899j | 0.00157+0.03786j | 0.05244-0.01191j | 0.05384-0.01636j | 0.10757+0.00000j | -0.04635+0.02734j | 0.00041+0.00097j
0.00559+0.02202j | -0.03789-0.00030j | -0.03091-0.04776j | -0.

In [10]:
head = data["head"]
head = head[:, :, 0, 0, 0, 0]

for row in range(head.shape[0]):
    row_data = []
    for col in range(head.shape[1]):
        row_data.append(f"{head[row, col]:.5f}")
    print(" | ".join(row_data))

1.00000-0.00000j | -0.05016-0.19131j | -0.11326-0.03814j | -0.12329+0.21733j | -0.17991+0.27518j | 0.02843-0.09821j | 0.04308+0.15990j | -0.00000+0.00000j
-0.05016-0.19131j | 1.00000+0.00000j | 0.04481-0.06806j | -0.24097-0.23319j | 0.05318+0.05884j | -0.28182-0.17010j | -0.03101+0.00090j | -0.00024+0.00175j
-0.11326-0.03814j | 0.04481-0.06806j | 1.00000-0.00000j | 0.06072-0.32644j | -0.02049+0.07906j | 0.00142+0.03097j | 0.18195+0.27839j | 0.00017-0.00008j
-0.12329+0.21733j | -0.24097-0.23319j | 0.06072-0.32644j | 1.00000-0.00000j | 0.03087+0.00288j | -0.04309+0.00855j | 0.03204-0.02999j | 0.01672+0.02048j
-0.17991+0.27518j | 0.05318+0.05884j | -0.02049+0.07906j | 0.03087+0.00288j | 1.00000+0.00000j | -0.31441+0.09672j | 0.22525-0.24866j | -0.00121-0.00144j
0.02843-0.09821j | -0.28182-0.17010j | 0.00142+0.03097j | -0.04309+0.00855j | -0.31441+0.09672j | 1.00000+0.00000j | 0.03747-0.02293j | 0.00345-0.00052j
0.04308+0.15990j | -0.03101+0.00090j | 0.18195+0.27839j | 0.03204-0.02999j | 0

In [11]:
wings = data["wings"]
wings = wings[:, :, 0, 0, 0, 0]

for row in range(wings.shape[0]):
    row_data = []
    for col in range(wings.shape[1]):
        row_data.append(f"{wings[row, col]:.5f}")
    print(" | ".join(row_data))

0.00000+0.00000j | -0.03187-0.12155j | -0.07195-0.02424j | 0.02096-0.03695j | -0.11430+0.17482j | -0.00482+0.01671j | -0.00733-0.02718j | -0.00000+0.00000j
-0.03187-0.12155j | 0.00000+0.00000j | 0.00749-0.01138j | -0.11627-0.11213j | 0.00890+0.00985j | -0.13604-0.08147j | 0.00650-0.00020j | -0.00011+0.00079j
-0.07195-0.02424j | 0.00749-0.01138j | 0.00000+0.00000j | 0.02952-0.15721j | -0.00342+0.01322j | -0.00029-0.00650j | 0.08784+0.13399j | 0.00004-0.00002j
0.02096-0.03695j | -0.11627-0.11213j | 0.02952-0.15721j | 0.00000+0.00000j | -0.00647-0.00062j | -0.00130+0.00026j | 0.00097-0.00091j | -0.00158-0.00282j
-0.11430+0.17482j | 0.00890+0.00985j | -0.00342+0.01322j | -0.00647-0.00062j | 0.00000+0.00000j | -0.15163+0.04606j | 0.10869-0.11963j | -0.00051-0.00061j
-0.00482+0.01671j | -0.13604-0.08147j | -0.00029-0.00650j | -0.00130+0.00026j | -0.15163+0.04606j | 0.00000+0.00000j | 0.00113-0.00069j | 0.00144-0.00072j
-0.00733-0.02718j | 0.00650-0.00020j | 0.08784+0.13399j | 0.00097-0.00091

In [12]:
body = data["body"]
body = body[:, :, 0, 0, 0, 0]

for row in range(body.shape[0]):
    row_data = []
    for col in range(body.shape[1]):
        row_data.append(f"{body[row, col]:.5f}")
    print(" | ".join(row_data))

0.32262-0.00000j | -0.09607-0.36649j | -0.21323-0.07183j | -0.24369+0.44140j | -0.35139+0.53751j | 0.05712-0.19565j | 0.07972+0.32109j | 0.06380-0.00767j
-0.09607-0.36649j | 0.24751+0.00000j | 0.22121-0.33597j | -0.38123-0.36056j | 0.26673+0.29510j | -0.44195-0.26461j | 0.10499-0.00071j | -0.02877+0.20851j
-0.21323-0.07183j | 0.22121-0.33597j | 0.24752-0.00000j | 0.09184-0.51144j | -0.10100+0.38977j | -0.00454-0.10489j | 0.27912+0.43929j | 0.00649-0.00309j
-0.24369+0.44140j | -0.38123-0.36056j | 0.09184-0.51144j | 0.17708+0.00000j | -0.10466-0.00838j | 0.30913-0.06897j | -0.23037+0.21710j | -0.05505-0.07055j
-0.35139+0.53751j | 0.26673+0.29510j | -0.10100+0.38977j | -0.10466-0.00838j | 0.24751-0.00000j | -0.49309+0.14790j | 0.35914-0.38305j | -0.13158-0.15662j
0.05712-0.19565j | -0.44195-0.26461j | -0.00454-0.10489j | 0.30913-0.06897j | -0.49309+0.14790j | 0.17695+0.00000j | -0.27250+0.16157j | -0.00906+0.00073j
0.07972+0.32109j | 0.10499-0.00071j | 0.27912+0.43929j | -0.23037+0.21710j

In [13]:
l_epsinv = epsilon.l_epsinv
eps0mat = l_epsinv[0]

# for row in range(0, eps0mat.shape[0]):
#     for col in range(0, eps0mat.shape[1]):
#         print(f"eps0mat[{row}, {col}] = {eps0mat[row, col]}")


In [14]:
l_epsinv_sorted = kernelclass.l_epsinv
eps0mat_sorted = l_epsinv_sorted[0]

# for row in range(0, eps0mat_sorted.shape[0]):
#     for col in range(0, eps0mat_sorted.shape[1]):
#         print(f"eps0mat_sorted[{row}, {col}] = {eps0mat_sorted[row, col]}")